## Ingestion calendrier paiements

In [1]:
%config Completer.use_jedi = False

In [2]:
import sys 
sys.path.append('/home/cdsw/code')
import imp
import ast
import numpy as np
import math
import openpyxl
import plotly.graph_objects as go
from plotly.colors import n_colors
import pandas as pd
from pyspark.sql import SparkSession
import dateutil
from dateutil.parser import parse
import configparser
import warnings
import dateutil
from dateutil.parser import parse
warnings.filterwarnings("ignore")



In [3]:
pd.set_option("max_rows", 500)

In [4]:
import Prev.multiple_sheet_process
import Prev.single_sheet_process
import utils
imp.reload(utils)
imp.reload(Prev.multiple_sheet_process)
imp.reload(Prev.single_sheet_process)
from utils import *
from Prev.multiple_sheet_process import * 
from Prev.single_sheet_process import * 

In [5]:
emplacement = "/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie"
extension = "*.xlsx"
list_filename = get_files_in_directory(emplacement,extension)


In [6]:
df = pd.read_excel('/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions - 2022 - ET 2021 10.xlsx', 
                   sheet_name = 'Calendrier avec échéancier' , 
                   engine = 'openpyxl')
df.columns = range(len(df.columns))

In [7]:
def extend_label(df,col):
    """
    params:
        df : dataframe of entry
        col : 
    returns : dataframe where the headers are extended to cells where nan values 
    """
    
    labels = df.loc[(df[col].isna()) & (df[col-1].notna()) ][col - 1].tolist()
    labels = [e for e in labels if type(e) == str]
    df.loc[(df[col -1].isin(labels)) & (df[col].isna()) , col] = labels

    return df
    

In [8]:
for c in range(1,len(df.columns)): 
    df = extend_label(df, c)

In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Acoss,Acoss,Acoss,Acoss,Acoss,Acoss,Acoss,Ministère Outre-Mer\n(LODEOM),Ministère Outre-Mer\n(LODEOM),Ministère Transition Ecologique\n(Armement Mar...,Ministère Transition Ecologique\n(Armement Mar...,Ministère de l'Agriculture\n(TO-DE),Ministère de l'Agriculture\n(TO-DE),Ministère du Travail\n(Aide à Domicile),Ministère du Travail\n(Aide à Domicile),Ministère du Travail\n(Apprentissage),Ministère du Travail\n(Apprentissage),Pôle emploi,Pôle emploi,Pôle emploi
1,TREC,TREC,CSG activité,CSG activité,Réduction \ngénérale,Acoss,CCMSA,Exonérations \nspécifiques,Exonérations \nspécifiques,Exonérations \nspécifiques,Exonérations \nspécifiques,Exonérations \nspécifiques,Exonérations \nspécifiques,Exonérations \nspécifiques,Exonérations \nspécifiques,Exonérations \nspécifiques,Exonérations \nspécifiques,Contribution (11%),Contribution (11%),Contribution (11%)
2,Acoss - TREC,Acoss - TREC,Acoss - CSG activité,Acoss - CSG activité,Acoss - CSG activité,Acoss - Réduction générale Acoss,Acoss - Réduction générale CCMSA,Ministère de l'Outre-mer - Exo. spé.,Ministère de l'Outre-mer - Exo. spé.,Ministère de la Transition écologique - Exo. spé.,Ministère de la Transition écologique - Exo. spé.,Ministère de l'Agriculture - Exo. spé.,Ministère de l'Agriculture - Exo. spé.,Ministère du Travail - Exo. spé. Aide à domicile,Ministère du Travail - Exo. spé. Aide à domicile,Ministère du Travail - Exo. spé. Apprentissage,Ministère du Travail - Exo. spé. Apprentissage,Pôle emploi - Participation Unédic (11%),Pôle emploi - Participation Unédic (11%),Pôle emploi - Participation Unédic (11%)
3,2022-01-05 00:00:00,1.021e+09,2022-01-05 00:00:00,5.62e+08,2022-01-05 00:00:00,96000000,7000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,2022-01-05 00:00:00,2e+08,NaN
4,2022-01-18 00:00:00,1.033e+09,2022-01-18 00:00:00,5.69e+08,2022-01-18 00:00:00,97000000,7000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,2022-01-25 00:00:00,1.27011e+08,NaN


In [10]:
def is_number(e):
    try :
        e = float(e)
        return True
    except :
        return False

In [11]:
# return if couple of cells is an observation
def cell_is_datapoint(df,c,i) :
    status = False
    """
    params
        df: dataframe  
        c : index of column
        i : index of row
    returns if the the cell composed of the column c and the row i and the cell at its right consist an observation 
    """
    
    if (type(df.iloc[i][df.columns[c]]) == datetime ) & is_number(df.iloc[i][df.columns[c+1]]) :
        status = True
        values = [c+1]
        try : 
            if is_number(df.iloc[i][df.columns[c+2]]):
                values = [c+1, c+2]
        except:
            pass
    else :
        status = False
        values = []
    
    return status, values
        
    

In [12]:
def get_level(df,c):
    return [e for e in df[df.columns[c]].tolist() if type(e) == str]

In [13]:
def ingestion_calendrier(filename, sheet_n, extension, type_montant):
    """
    params
    filename : the file's name to ingest
    sheet_n : the sheet_name to tranform into data table
    extension : the extension of the file to ingest 
    type_montant : wether it is an observation values or a prediction 
    
    return : result dataframe
    
    """
    
    ## read the file to ingest
    if extension == '.xlsx':
        df = pd.read_excel(filename, sheet_name = sheet_n, engine = 'openpyxl')
    elif extension == '.csv' : 
        df = pd.read_csv(filename, sheet_name = sheet_n, engine = 'openpyxl' ) 
        
    df.columns = range(len(df.columns))
    df.replace(to_replace=[None], value=np.nan, inplace=True)

    ## create the output dataframe
    calendrier = pd.DataFrame(columns = ['date','type de calendrier','opérateur', 'nature_niv01','nature_niv02','montant'])
        
    # extend the header labels all over data 
    for c in range(1,len(df.columns)): 
        df = extend_label(df, c)
    
      
    for col in df.columns-1 :
        for i in range(len(df)):
            status, values = cell_is_datapoint(df,col,i)
            if status:
                for v in values: 
                    level = get_level(df, col)
                    calendrier = calendrier.append({"date" : df.iloc[i][col] , 'type de calendrier' : type_montant ,
                        'nature_niv02': level[2],"opérateur" : level[0] , "nature_niv01" : level[1],
                        "montant": df.iloc[i][v] } ,
                                      ignore_index = True)
                    calendrier = calendrier.drop_duplicates()
                    calendrier = calendrier.dropna()

    return calendrier

In [14]:
filename = '/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions - 2022 - ET 2021 10.xlsx'
sheet_n = 'Calendrier avec échéancier'

calendrier_échéancier = ingestion_calendrier(filename, sheet_n, '.xlsx', 'écheancier')

In [15]:
sheet_n1 = 'Calendrier (estimation)'

calendrier_estimation = ingestion_calendrier(filename, sheet_n1, '.xlsx', 'estimation')

In [16]:
calendrier_échéancier

,date,type de calendrier,opérateur,nature_niv01,nature_niv02,montant
0,2022-01-05,écheancier,Acoss,TREC,Acoss - TREC,1.021000e+09
1,2022-01-18,écheancier,Acoss,TREC,Acoss - TREC,1.033000e+09
2,2022-02-07,écheancier,Acoss,TREC,Acoss - TREC,8.050000e+08
3,2022-02-16,écheancier,Acoss,TREC,Acoss - TREC,6.630000e+08
4,2022-03-07,écheancier,Acoss,TREC,Acoss - TREC,7.570000e+08
5,2022-03-16,écheancier,Acoss,TREC,Acoss - TREC,6.770000e+08
6,2022-04-05,écheancier,Acoss,TREC,Acoss - TREC,8.580000e+08
7,2022-04-19,écheancier,Acoss,TREC,Acoss - TREC,8.490000e+08
8,2022-05-05,écheancier,Acoss,TREC,Acoss - TREC,7.680000e+08
9,2022-05-17,écheancier,Acoss,TREC,Acoss - TREC,8.940000e+08


### chargement des données sous table Hive

In [17]:
applicationName = "chargement_calendrier"
spark = SparkSession\
        .builder\
        .appName(applicationName)\
        .getOrCreate()

In [18]:
depot = "/data/raw_layer"
db_name = "raw_reporting_treso"
table_name= "calendrier_dft"


In [19]:
process = subprocess.Popen(["hadoop", "fs", "-rm","-r","-skipTrash", depot+"/"+db_name+"/"+table_name],stdout=subprocess.PIPE)
process.wait()
spark.sql("DROP TABLE IF EXISTS "+db_name+"."+table_name)

DataFrame[]

In [22]:
df= calendrier_estimation

df.columns = [traiter_string(s).upper() for s in df.columns]
#replace the \n values 
df = df[df.columns].astype(str).replace(to_replace='\n', value=' ', regex=True)

df

,DATE,TYPE_DE_CALENDRIER,OPERATEUR,NATURE_NIV01,NATURE_NIV02,MONTANT
0,2022-01-10,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0
1,2022-02-10,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0
2,2022-03-10,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0
3,2022-04-11,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0
4,2022-05-10,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0
5,2022-06-10,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0
6,2022-07-11,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9547033.11
7,2022-08-10,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0
8,2022-09-12,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0
9,2022-10-10,estimation,Monaco,Recouvrement des contributions,Caisse Sociale de Monaco - Contributions,9000000.0


In [23]:


## create the Spark DataFrame
spark_data_frame = spark.createDataFrame(df)

#spark_data_frame.show(200,truncate = False)
spark_data_frame.show()
spark_data_frame.printSchema()

## Create Database if doesn't exist 
spark.sql("CREATE DATABASE IF NOT EXISTS "+db_name+" LOCATION '"+depot+"/"+db_name+"'")
#---------------------------------------------------------------------------
# ecriture en mode overwrite plutot qu'append pour ne pas être en mode incremental
#---------------------------------------------------------------------------
spark_data_frame.write.mode('append').parquet(depot+"/"+db_name+"/"+table_name)
spark.sql("CREATE TABLE IF NOT EXISTS "+db_name+"."+table_name+" USING PARQUET LOCATION '"+depot+"/"+db_name+"/"+table_name+"'")

+----------+------------------+---------+--------------------+--------------------+----------+
|      DATE|TYPE_DE_CALENDRIER|OPERATEUR|        NATURE_NIV01|        NATURE_NIV02|   MONTANT|
+----------+------------------+---------+--------------------+--------------------+----------+
|2022-01-10|        estimation|   Monaco|Recouvrement des ...|Caisse Sociale de...| 9000000.0|
|2022-02-10|        estimation|   Monaco|Recouvrement des ...|Caisse Sociale de...| 9000000.0|
|2022-03-10|        estimation|   Monaco|Recouvrement des ...|Caisse Sociale de...| 9000000.0|
|2022-04-11|        estimation|   Monaco|Recouvrement des ...|Caisse Sociale de...| 9000000.0|
|2022-05-10|        estimation|   Monaco|Recouvrement des ...|Caisse Sociale de...| 9000000.0|
|2022-06-10|        estimation|   Monaco|Recouvrement des ...|Caisse Sociale de...| 9000000.0|
|2022-07-11|        estimation|   Monaco|Recouvrement des ...|Caisse Sociale de...|9547033.11|
|2022-08-10|        estimation|   Monaco|Recouvrem

DataFrame[]